In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

In [17]:
artists = ["coldplay"]

In [23]:
# theaudiodb.com/api/v1/json/1/search.php?s=coldplay

audio_data = {'artist':[],
              'style':[], 
              'genre':[]
             }

#loop over artists dictionary and add data to dictionary
for artist in artists:
    base_url = "theaudiodb.com/api/v1/json/1/search.php?s="

    try:
        req = requests.get(base_url + f'{artist}')
        data = req.json()
        
        #add values to dictionary
        audio_data['artist'].append(data["strArtist"])
        audio_data['style'].append(data["strStyle"])
        audio_data['genre'].append(data["strGenre"])

    except:
        pass

In [21]:
print(audio_data)

{'artist': [], 'style': [], 'genre': []}
